In [2]:
import cv2
import pandas as pd
import numpy as np

def preprocessing(val=True):
    
    data = pd.read_csv('data/x_train_gr_smpl.csv')
    labels = pd.read_csv('data/y_train_smpl.csv')

    if(val):
        ## ---------------- Data preparation ---------------- ##
        X_train = []
        for i in range(data.shape[0]):
            img = np.uint8(data.iloc[i])
            edited = cv2.Canny(img, 10, 30)
            edited = cv2.GaussianBlur(edited, (5, 5), 0)
            X_train.append(edited.reshape((1,-1))[0])

        data = pd.DataFrame(X_train)
        ## -------------------------------------------------- ##

    return data, labels

In [ ]:
data, labels = preprocessing()
data['label'] = labels
data = data.sample(frac=1)

data['label'] = data['label'].map({0: 'zero', 1: 'one', 2: 'two', 3: 'three', 4: 'four', 5: 'five', 6: 'six', 7: 'seven', 8: 'eight', 9: 'nine'})
data.to_csv('DecisionTrees/src/main/resources/data_gr.csv', encoding='utf-8', sep=',', index=False)

In [ ]:
# Generate reduced dataset to work with Weka

top_2_array = set()
top_5_array = set()
top_10_array = set()

data, _ = preprocessing()

for i in range(10):
    data = data.reindex(np.arange(data.shape[0]))
    labels = pd.read_csv(f'data/y_train_smpl_{i}.csv')
    data['label'] = labels
    data = data.sample(frac=1)
    
    corr_label = data.drop("label", axis=1).apply(lambda x: x.corr(data.label))
    corr_label = [(index, abs(corr_val), i) for index, corr_val in enumerate(corr_label)]
    corr_label = sorted(corr_label, key=lambda tup: tup[1], reverse=True)  # Order by correlation value
            
    for i, tup in enumerate(corr_label[:10]):
        if i < 2:
            top_2_array.add(tup[0])
        if i < 5:
            top_5_array.add(tup[0])
        if i < 10:
            top_10_array.add(tup[0])
        
data, labels = preprocessing()

In [ ]:
data_top_10 = data[data.columns[list(top_10_array)]].copy(deep=True)
data_top_10['label'] = labels
data_top_10['label'] = data_top_10['label'].map({0: 'zero', 1: 'one', 2: 'two', 3: 'three', 4: 'four', 5: 'five', 6: 'six', 7: 'seven', 8: 'eight', 9: 'nine'})
data_top_10 = data_top_10.sample(frac=1).reset_index(drop=True)
data_top_10.to_csv('DecisionTrees/src/main/resources/data_top_10.csv', encoding='utf-8', sep=',', index=False)